In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install torch torchvision matplotlib scikit-learn tqdm timm -q

In [3]:
import os
import time
import copy
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import transforms, datasets, models

from sklearn.metrics import classification_report, confusion_matrix, f1_score, accuracy_score
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", device)

Device: cpu


In [5]:
# Paths
train_dir = "/content/drive/MyDrive/DL/train"
val_dir   = "/content/drive/MyDrive/DL/val"
test_dir  = "/content/drive/MyDrive/DL/test"

# Data transforms
train_transforms = transforms.Compose([
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(20),
    transforms.ColorJitter(brightness=0.1, contrast=0.1, saturation=0.1),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])

val_test_transforms = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])

# Datasets
train_dataset = datasets.ImageFolder(train_dir, transform=train_transforms)
val_dataset   = datasets.ImageFolder(val_dir, transform=val_test_transforms)
test_dataset  = datasets.ImageFolder(test_dir, transform=val_test_transforms)

# OPTIONAL: If you want to exclude one class (like "Melanocytic nevi")
exclude_class = "Melanocytic nevi"
if exclude_class in train_dataset.class_to_idx:
    exclude_idx = train_dataset.class_to_idx[exclude_class]
    from torch.utils.data import Subset
    train_indices = [i for i, (_, label) in enumerate(train_dataset.samples) if label != exclude_idx]
    val_indices = [i for i, (_, label) in enumerate(val_dataset.samples) if label != exclude_idx]
    test_indices = [i for i, (_, label) in enumerate(test_dataset.samples) if label != exclude_idx]
    train_dataset = Subset(train_dataset, train_indices)
    val_dataset = Subset(val_dataset, val_indices)
    test_dataset = Subset(test_dataset, test_indices)
    class_names = [cls for cls in datasets.ImageFolder(train_dir).classes if cls != exclude_class]
else:
    class_names = train_dataset.classes

# DataLoaders
batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2)
val_loader   = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=2)
test_loader  = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=2)

# Class info
num_classes = len(class_names)
print(f"Classes ({num_classes}):", class_names)
print("Train size:", len(train_dataset))
print("Val size:", len(val_dataset))
print("Test size:", len(test_dataset))


Classes (13): ['Actinic keratoses', 'Basal cell carcinoma', 'Benign keratosis-like lesions', 'Chickenpox', 'Cowpox', 'Dermatofibroma', 'HFMD', 'Healthy', 'Measles', 'Melanoma', 'Monkeypox', 'Squamous cell carcinoma', 'Vascular lesions']
Train size: 19022
Val size: 2373
Test size: 2386


In [6]:
import torch
import torch.nn as nn
from collections import Counter

# Determine the device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# --- Get labels from the train dataset ---
if isinstance(train_dataset, torch.utils.data.Subset):
    # For Subset, use the original dataset and indices
    labels = [train_dataset.dataset.samples[i][1] for i in train_dataset.indices]
else:
    labels = [y for _, y in train_dataset.samples]

# Compute class counts
counts = Counter(labels)
counts_list = [counts[i] for i in range(len(class_names))]
print("Class counts:", counts_list)

# Compute class weights (inverse frequency)
class_weights = torch.tensor([sum(counts_list)/c for c in counts_list], dtype=torch.float).to(device)
print("Class weights:", class_weights)

# Define weighted CrossEntropyLoss
criterion = nn.CrossEntropyLoss(weight=class_weights)

Class counts: [693, 2658, 2099, 900, 792, 191, 1932, 1368, 660, 3617, 3408, 502, 202]
Class weights: tensor([27.4488,  7.1565,  9.0624, 21.1356, 24.0177, 99.5916,  9.8458, 13.9050,
        28.8212,  5.2591,  5.5816, 37.8924, 94.1683])


In [8]:
from torchvision.models import resnet50, ResNet50_Weights
import torch.nn as nn

# Load ResNet-50 with the latest API
weights = ResNet50_Weights.DEFAULT  # or IMAGENET1K_V1
model = resnet50(weights=weights)

# Replace the final layer for your num_classes
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, num_classes)

# Move to device
model = model.to(device)

Downloading: "https://download.pytorch.org/models/resnet50-11ad3fa6.pth" to /root/.cache/torch/hub/checkpoints/resnet50-11ad3fa6.pth


100%|██████████| 97.8M/97.8M [00:00<00:00, 110MB/s]


In [9]:
import torch.optim as optim
from torch.optim import lr_scheduler

# Optimizer: Adam with weight decay
optimizer = optim.Adam(model.parameters(), lr=1e-4, weight_decay=1e-5)

# Optional learning rate scheduler (uncomment if you want to use it)
# StepLR: reduce LR by gamma every step_size epochs
# scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

# Currently no scheduler
scheduler = None

# Number of training epochs
num_epochs = 10  # start with 10; increase if model hasn't converged